# Day 0: Imports and Utility Functions

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import re
import numpy as np
import random
import string
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools   import lru_cache, reduce
from statistics  import mean, median, mode, stdev, variance
from itertools   import (permutations, combinations, groupby, cycle, chain, zip_longest, takewhile, dropwhile, count as count_from)
from heapq       import heappush, heappop, nsmallest
from operator    import iand, ior, ilshift, irshift

# Day 1: No Time for a Taxicab

In [2]:
with open("inputs/day1.txt", 'r') as f:
    dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    orders = f.read().split(', ')
    x, y = 0, 0
    idx = 0
    repeat = None
    positions = {(x, y)}
    for order in orders:
        d, num = order[0], int(order[1:])
        if d == 'L':
            idx -= 1
            if idx < 0:
                idx = 3
        else:
            idx += 1
            if idx > 3:
                idx = 0
        dx, dy = dirs[idx]
        for i in range(num):
            x, y = x + dx, y + dy
            if (x, y) in positions:
                if repeat is None:
                    repeat = (x, y)
            else:
                positions.add((x, y))
    print(abs(x) + abs(y))
    print(repeat)

300
(9, -150)


# Day 2: Bathroom Security

In [3]:
grid = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
dirs = dict(U=(-1, 0), D=(1, 0), L=(0, -1), R=(0, 1))

In [4]:
with open('inputs/day2.txt', 'r') as f:
    i, j = 1, 1
    ans = []
    m, n = len(grid), len(grid[0])
    for line in f.readlines():
        for ch in line.strip():
            di, dj = dirs[ch]
            i, j = i + di, j + dj
            if not (0 <= i < m and 0 <= j < n):
                i, j = i - di, j - dj
        ans.append(grid[i][j])
    print(''.join(map(str, ans)))

33444


In [5]:
grid = [[0, 0, 1, 0, 0], [0, 2, 3, 4, 0], [5, 6, 7, 8, 9], [0, 'A', 'B', 'C', 0], [0, 0, 'D', 0, 0]]
with open('inputs/day2.txt', 'r') as f:
    i, j = 2, 0
    ans = []
    m, n = len(grid), len(grid[0])
    for line in f.readlines():
        for ch in line.strip():
            di, dj = dirs[ch]
            i, j = i + di, j + dj
            if not (0 <= i < m and 0 <= j < n) or grid[i][j] == 0:
                i, j = i - di, j - dj
        ans.append(grid[i][j])
    print(''.join(map(str, ans)))

446A6


# Day 3: Squares With Three Sides

In [6]:
def is_triangle(triangle):
    s = sum(triangle)
    m = max(triangle)
    return s - m > m

In [7]:
with open('inputs/day3.txt', 'r') as f:
    print(sum(is_triangle(tuple(map(int, line.split()))) for line in f))

917


In [8]:
m = np.loadtxt('inputs/day3.txt').flatten('F') # flatten 

In [9]:
m = m.reshape((len(m)//3, 3))

In [10]:
print(sum(is_triangle(tuple(row)) for row in m))

1649


# Day 4: Security Through Obscurity

In [11]:
with open('inputs/day4.txt', 'r') as f:
    ans = 0
    for line in f.readlines():
        line = line.strip()
        letters, id_check = line.rsplit('-', 1)
        counter = Counter(letters.replace('-', ''))
        pairs = [(-cnt, ch) for ch, cnt in counter.items() if ch.isalpha()]
        checksum = ''.join(ch for _, ch in heapq.nsmallest(5, pairs))
        if checksum == id_check[-6:-1]:
            ans += int(id_check[:-7])
print(ans)

361724


In [12]:
with open('inputs/day4.txt', 'r') as f:
    L = string.ascii_lowercase
    for line in f:
        line = line.strip()
        letters, id_check = line.rsplit('-', 1)
        ID = int(id_check[:-7])
        move = ID % 26
        letters = list(letters)
        for i, ch in enumerate(letters):
            if ch.isalpha():
                pos = ord(ch) - 97
                pos = (pos + move) % 26
                letters[i] = L[pos]
            else:
                letters[i] = ' '
        letters = ''.join(letters)
        if "north" in letters:
            print(ID)

482


# Day 5: How About a Nice Game of Chess?

In [13]:
import hashlib
s = "wtnhxymk"
i = 0
ans = ''
while len(ans) < 8:
    m = hashlib.md5()
    m.update((s + str(i)).encode('utf-8'))
    code = m.hexdigest()
    if code.startswith('00000'):
        ans += code[5]
    i += 1
print(ans)

2414bc77


In [14]:
s = "wtnhxymk"
i = 0
ans = [None] * 8
cnt = 0
while cnt < 8:
    m = hashlib.md5()
    m.update((s + str(i)).encode('utf-8'))
    code = m.hexdigest()
    if code.startswith('00000'):
        if code[5].isdigit() and int(code[5]) < 8 and ans[int(code[5])] is None:
            ans[int(code[5])] = code[6]
            cnt += 1
    i += 1
print(ans)

['4', '3', '7', 'e', '6', '0', 'f', 'c']


In [15]:
print(''.join(ans))

437e60fc


# Day 6: Signals and Noise

In [16]:
counters = [Counter() for _ in range(8)]
with open('inputs/day6.txt', 'r') as f:
    for line in f:
        for counter, ch in zip(counters, line):
            counter[ch] += 1
print(''.join([counter.most_common(1)[0][0] for counter in counters]))
print(''.join([counter.most_common()[-1][0] for counter in counters]))

mshjnduc
apfeeebz


# Day 7: Internet Protocol Version 7

In [17]:
def support_TLS(line):
    has = False
    for i in range(len(line) - 3):
        four = line[i:i + 4]
        if four[0] == four[3] and four[1] == four[2] and four[0] != four[1]:
            j = i - 1
            while j >= 0 and line[j] != ']':
                if line[j] == '[':
                    return False
                j -= 1
            has = True
    return has

In [18]:
with open('inputs/day7.txt', 'r') as f:
    print(sum(support_TLS(line) for line in f))

105


In [19]:
def support_SSL(line):
    outs = set()
    ins = set()
    for i in range(len(line) - 2):
        three = line[i:i + 3]
        if three[0] == three[2] != three[1]:
            j = i - 1
            while j >= 0 and line[j] != ']':
                if line[j] == '[':
                    ins.add(three)
                    break
                j -= 1
            else:
                outs.add(three)
    return any((three[1] + three[0] + three[1]) in outs for three in ins)
with open('inputs/day7.txt', 'r') as f:
    print(sum(support_SSL(line) for line in f))

258


# Day 8: Two-Factor Authentication

In [20]:
m, n = 6, 50
grid = [[0] * n for _ in range(m)]
with open('inputs/day8.txt', 'r') as f:
    for i, line in enumerate(f):
#         print(i)
        if line.startswith('rect'):
            _, item = line.split()
            c, r = item.split('x')
            for i in range(int(r)):
                for j in range(int(c)):
                    grid[i][j] = 1
        else:
            *_, rc, _, num = line.split()
            num = int(num)
            if rc[0] == 'y':
                # rotate row
                r = int(rc[2:])
                num = num % n
                if num != 0:
                    grid[r] = grid[r][-num:] + grid[r][:-num]
            else:
                # rotate col
                c =int(rc[2:])
                col = [grid[i][c] for i in range(m)]
                num = num % m
                if num != 0:
                    col = col[-num:] + col[:-num]
                    for i in range(m):
                        grid[i][c] = col[i]
print(sum(map(sum, grid)))

119


In [21]:
len(grid), len(grid[0])

(6, 50)

In [22]:
cols = zip(*grid)
iters = [iter(cols)] * 5
code = list(zip(*iters))
[sum(map(sum, item)) for item in code]

[12, 11, 14, 11, 11, 11, 12, 13, 12, 12]

In [23]:
for letter in code:
    for row in zip(*letter):
        print(row)
    print()

(1, 1, 1, 1, 0)
(0, 0, 0, 1, 0)
(0, 0, 1, 0, 0)
(0, 1, 0, 0, 0)
(1, 0, 0, 0, 0)
(1, 1, 1, 1, 0)

(1, 1, 1, 1, 0)
(1, 0, 0, 0, 0)
(1, 1, 1, 0, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)

(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(1, 1, 1, 1, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)

(1, 1, 1, 1, 0)
(1, 0, 0, 0, 0)
(1, 1, 1, 0, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)

(0, 1, 1, 1, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)
(0, 1, 1, 0, 0)
(0, 0, 0, 1, 0)
(1, 1, 1, 0, 0)

(1, 1, 1, 1, 0)
(1, 0, 0, 0, 0)
(1, 1, 1, 0, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)

(0, 1, 1, 0, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(0, 1, 1, 0, 0)

(0, 1, 1, 0, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 0, 0)
(1, 0, 1, 1, 0)
(1, 0, 0, 1, 0)
(0, 1, 1, 1, 0)

(1, 1, 1, 0, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(1, 1, 1, 0, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)

(0, 1, 1, 0, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(0, 1, 1, 0, 0)



# Day 9: Explosives in Cyberspace

In [24]:
def decompress_v1(line):
    l = line.find('(')
    if l != -1:
        r = line.find(')', l)
        length, num = line[l+1:r].split('x')
        num = int(num)
        length = int(length)
        return l + length * num + decompress_v1(line[r + 1 + length:])
    else:
        return len(line)
def decompress_v2(line):
    l = line.find('(')
    if l != -1:
        r = line.find(')', l)
        length, num = line[l+1:r].split('x')
        num = int(num)
        length = int(length)
        return l + num * decompress_v2(line[r+1:r+1+length]) + decompress_v2(line[r + 1 + length:])
    else:
        return len(line)
with open('inputs/day9.txt', 'r') as f:
    line = f.read()
print(decompress_v1(line))
print(decompress_v2(line))

183269
11317278863


# Day 10: Balance Bots

In [25]:
bots = defaultdict(list)
operators = defaultdict(list)
bins = defaultdict(list)
with open('inputs/day10.txt', 'r') as f:
    lines = f.readlines()
    inits = [line.strip() for line in lines if line.startswith('value')]
    ops = [line.strip() for line in lines if line.startswith('bot')]
for init in inits:
    _, num, *_, bot_num = init.split()
    bots[int(bot_num)].append(int(num))
for key in bots:
    bots[key].sort()
for op in ops:
    items = op.split()
    assert(int(items[1]) not in operators)
    operators[int(items[1])] = [items[5:7], items[-2:]]

In [26]:
def bot_op(dest, dest_num, num):
    if dest == 'output':
        bins[dest_num].append(num)
    else:
        bots[dest_num].append(num)
        bots[dest_num].sort()
flag = True
while flag:
    flag = False
    for bot_num in bots:
        if len(bots[bot_num]) == 2 and bot_num in operators:
            assert(bot_num in operators)
            low, high = operators.pop(bot_num)
            low, low_dest = low
            high, high_dest = high
            low_num, high_num = bots[bot_num]
            if low_num == 17 and high_num == 62:
                print(bot_num)
            bot_op(low, int(low_dest), low_num)
            bot_op(high, int(high_dest), high_num)
            flag = True
            break

In [27]:
for key in bots:
    if bots[key] == [17, 61]:
        print(key)

86


In [28]:
bins[0][0] * bins[1][0] * bins[2][0]

22847

# Day 11: Radioisotope Thermoelectric Generators